In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import layers, models
from tensorflow.keras.saving import register_keras_serializable
import tensorflow as tf
import joblib

data = pd.read_csv("dataset/hurricane_data.csv")
X = data.drop("hurricane_binary", axis=1).astype("float32")

def generate_trust_score(row):
    sst     = row["sea_surface_temperature"]
    ohc     = row["ocean_heat_content"]
    humid   = row["mid_level_humidity"]
    shear   = row["vertical_wind_shear"]
    vort    = row["potential_vorticity"]

    sst_factor   = np.clip((sst - 26) / 5, 0, 1)
    ohc_factor   = np.clip((ohc - 60) / 60, 0, 1)
    humidity_factor = np.clip((humid - 40) / 40, 0, 1)
    shear_factor = np.clip((20 - shear) / 20, 0, 1)
    vort_factor  = np.clip((vort - 0.5) / 1.5, 0, 1)

    composite = (
        0.25 * sst_factor +
        0.25 * ohc_factor +
        0.2  * humidity_factor +
        0.2  * shear_factor +
        0.1  * vort_factor
    )
    return 0.5 + composite

trust_labels = X.apply(generate_trust_score, axis=1).astype("float32")

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
joblib.dump(scaler, "models/HurricaneTrustScaler.pkl")

@register_keras_serializable()
def trust_activation(x):
    return 0.5 + tf.sigmoid(x)

input_layer = layers.Input(shape=(5,))
x = layers.Dense(32, activation="relu")(input_layer)
x = layers.Dense(16, activation="relu")(x)
x = layers.Dense(8, activation="relu")(x)
trust_output = layers.Dense(1, activation=trust_activation)(x)

model = models.Model(inputs=input_layer, outputs=trust_output)
model.compile(optimizer="adam", loss="mse")
model.summary()

model.fit(X_scaled, trust_labels, validation_split=0.2, batch_size=32, epochs=10)

model.save("models/HurricaneTrustNet.h5")
import tf2onnx
import json
model_proto, _ = tf2onnx.convert.from_keras(model)
with open(f"models/ONNX/HurricaneTrustNet.onnx", "wb") as f:
    f.write(model_proto.SerializeToString())

scaler_data = {
        "mean": scaler.mean_.tolist(),
        "scale": scaler.scale_.tolist()
}
with open(f"models/SCALER-CFG/HurricaneScaler.json", "w") as f:
    json.dump(scaler_data, f)

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 5)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │           192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 865 (3.38 KB)

 Trainable params: 865 (3.38 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 10s 22ms/step - loss: 0.0600 - val_loss: 0.0014
Epoch 2/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.0013 - val_loss: 7.2611e-04
Epoch 3/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 5.9943e-04 - val_loss: 4.6340e-04
Epoch 4/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 4.0175e-04 - val_loss: 3.7479e-04
Epoch 5/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - loss: 3.2001e-04 - val_loss: 3.2390e-04
Epoch 6/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 2.8310e-04 - val_loss: 3.0178e-04
Epoch 7/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 2.8411e-04 - val_loss: 2.8106e-04
Epoch 8/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - loss: 2.5713e-04 - val_loss: 2.8654e-04
Epoch 9/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 2.4527e-04 - val_loss: 2.6941e-04
Epoch 10/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 2.2407e-04 - val_loss: 2.4949e-04



ERROR:tf2onnx.tfonnx:rewriter <function rewrite_constant_fold at 0x000001FC067BD3A0>: exception `np.cast` was removed in the NumPy 2.0 release. Use `np.asarray(arr, dtype=dtype)` instead.
